In [1]:
import os
import findspark

if os.path.isdir("/cluster/home/simondi/spark/"):
    is_cluster = True
    import sparkhpc
    from sparkhpc import sparkjob
    spark_path = "/cluster/home/simondi/spark/"
else:
    is_cluster = False
    spark_path = "/usr/local/spark/spark/"

findspark.init(spark_path)
import pyspark

In [2]:
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [3]:
conf = pyspark.SparkConf().setMaster("local[*]").setAppName("test").set("spark.driver.memory", "10G").set("spark.executor.memory", "5G")
sc = pyspark.SparkContext(conf=conf)

In [4]:
spark = pyspark.sql.SparkSession(sc)

In [18]:
if is_cluster:
    file_name = "/cluster/home/simondi/simondi/tix/data/screening_data/cells_sample_10.tsv"
else:
    file_name = "/Users/simondi/PHD/data/data/target_infect_x/screening_data_subset/cells_sample_10.tsv"
file_name    

'/cluster/home/simondi/simondi/tix/data/screening_data/cells_sample_10.tsv'

In [19]:
df = spark.read.csv(path=file_name, sep="\t", header='true')
df.cache()
df.take(1)

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:41480)

In [8]:
df.rdd.getNumPartitions()

48

In [9]:
old_cols = df.schema.names
new_cols = list(map(lambda x: x.replace(".", "_"),  old_cols))
df = reduce(lambda data, idx: data.withColumnRenamed(old_cols[idx], new_cols[idx]), range(len(new_cols)), df)
for i, x in enumerate(new_cols):
    if x.startswith("cells"):
        df= df.withColumn(x, df[x].cast("double"))

In [10]:
def z_score_w(col, w):
    avg = func.avg(col).over(w)
    sd = func.stddev(col).over(w)
    return (col - avg) / sd

In [15]:
df = df.sample(fraction=0.001, withReplacement=False)

Py4JError: An error occurred while calling o295.sample

In [11]:
import sys
w = Window().partitionBy(["study", "pathogen"]).rowsBetween(-sys.maxsize, sys.maxsize)
for x in df.columns:
    if x.startswith("cells"):
        df = df.withColumn(x, z_score_w(df[x], w))

In [12]:
df.take(1)

Py4JJavaError: An error occurred while calling o296.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 61 in stage 10.0 failed 1 times, most recent failure: Lost task 61.0 in stage 10.0 (TID 172, localhost, executor driver): java.lang.OutOfMemoryError: Unable to acquire 16384 bytes of memory, got 0
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:100)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeInMemorySorter.<init>(UnsafeInMemorySorter.java:126)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:154)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:121)
	at org.apache.spark.sql.execution.window.WindowExec$$anonfun$14$$anon$1.fetchNextPartition(WindowExec.scala:340)
	at org.apache.spark.sql.execution.window.WindowExec$$anonfun$14$$anon$1.next(WindowExec.scala:391)
	at org.apache.spark.sql.execution.windo

In [15]:
outfile = file_name.replace(".tsv", "") + "zscored.tsv"
df.write.option("sep","\t").option("header","true").csv(outfile)

Py4JJavaError: An error occurred while calling o347.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:147)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:101)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apache

In [22]:
sc.stop()